<a href="https://colab.research.google.com/github/waltz2u/bd/blob/master/Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# Setting things up

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,784

In [3]:
!ls

gdrive	sample_data  spark-2.4.5-bin-hadoop2.7	spark-2.4.5-bin-hadoop2.7.tgz


In [4]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [5]:
!java -version 

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Spark DataFrame

For the following to work, you must have installed pyspark on Anaconda beforehand

conda install -c conda-forge pyspark

Then change the path to the data folder accordingly

In [0]:
import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
# Sets the Spark master URL to connect to, such as "local" to run locally, 
# "local[4]" to run locally with 4 cores, or "spark://master:7077" to run on a Spark standalone cluster.
spark = SparkSession.builder.master("local[*]").getOrCreate() #

In [0]:
usage_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/device_usages/user_usage.csv')

First n observations

In [9]:
usage_df.head(5)

[Row(outgoing_mins_per_month='21.97', outgoing_sms_per_month='4.82', monthly_mb='1557.33', use_id='22787'),
 Row(outgoing_mins_per_month='1710.08', outgoing_sms_per_month='136.88', monthly_mb='7267.55', use_id='22788'),
 Row(outgoing_mins_per_month='1710.08', outgoing_sms_per_month='136.88', monthly_mb='7267.55', use_id='22789'),
 Row(outgoing_mins_per_month='94.46', outgoing_sms_per_month='35.17', monthly_mb='519.12', use_id='22790'),
 Row(outgoing_mins_per_month='71.59', outgoing_sms_per_month='79.26', monthly_mb='1557.33', use_id='22792')]

In [10]:
usage_df.show(5)

+-----------------------+----------------------+----------+------+
|outgoing_mins_per_month|outgoing_sms_per_month|monthly_mb|use_id|
+-----------------------+----------------------+----------+------+
|                  21.97|                  4.82|   1557.33| 22787|
|                1710.08|                136.88|   7267.55| 22788|
|                1710.08|                136.88|   7267.55| 22789|
|                  94.46|                 35.17|    519.12| 22790|
|                  71.59|                 79.26|   1557.33| 22792|
+-----------------------+----------------------+----------+------+
only showing top 5 rows



In [0]:
device_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/models/device_usages/user_device.csv')

In [12]:
device_df.show(3)

+------+-------+--------+----------------+---------+-----------+
|use_id|user_id|platform|platform_version|   device|use_type_id|
+------+-------+--------+----------------+---------+-----------+
| 22782|  26980|     ios|            10.2|iPhone7,2|          2|
| 22783|  29628| android|               6|  Nexus 5|          3|
| 22784|  28473| android|             5.1| SM-G903F|          1|
+------+-------+--------+----------------+---------+-----------+
only showing top 3 rows



Group by

In [13]:
device_df.groupby('platform').count().show()

+--------+-----+
|platform|count|
+--------+-----+
| android|  184|
|     ios|   88|
+--------+-----+



Another way: use agg() method for aggregation

In [14]:
device_df.groupby('platform').agg({"platform":"count"}).show(5)

+--------+---------------+
|platform|count(platform)|
+--------+---------------+
| android|            184|
|     ios|             88|
+--------+---------------+



In [15]:
usage_df.groupby('use_id').agg({"monthly_mb":"sum"}).show(5)

+------+---------------+
|use_id|sum(monthly_mb)|
+------+---------------+
| 22920|        7267.55|
| 22831|         407.01|
| 22922|        1557.33|
| 22875|        1557.33|
| 22895|        3114.67|
+------+---------------+
only showing top 5 rows



In [16]:
usage_df.groupby('use_id').agg({"monthly_mb":"max"}).show(5)

+------+---------------+
|use_id|max(monthly_mb)|
+------+---------------+
| 22920|        7267.55|
| 22831|         407.01|
| 22922|        1557.33|
| 22875|        1557.33|
| 22895|        3114.67|
+------+---------------+
only showing top 5 rows



Ordered by (multiple forms)

In [17]:
device_df.orderBy('platform',ascending = True).show(3)

+------+-------+--------+----------------+---------+-----------+
|use_id|user_id|platform|platform_version|   device|use_type_id|
+------+-------+--------+----------------+---------+-----------+
| 22786|  28239| android|               6|ONE E1003|          1|
| 22789|  28714| android|               6| SM-G930F|          1|
| 22787|  12921| android|             4.3| GT-I9505|          1|
+------+-------+--------+----------------+---------+-----------+
only showing top 3 rows



In [18]:
device_df.orderBy('platform','device').show(3)

+------+-------+--------+----------------+------+-----------+
|use_id|user_id|platform|platform_version|device|use_type_id|
+------+-------+--------+----------------+------+-----------+
| 22839|  29655| android|               6| A0001|          1|
| 22806|  21615| android|               6| A0001|          1|
| 23028|  29716| android|             5.1| C6603|          1|
+------+-------+--------+----------------+------+-----------+
only showing top 3 rows



In [19]:
device_df.orderBy(['platform','device'],ascending=[0,1]).show(3)

+------+-------+--------+----------------+-------+-----------+
|use_id|user_id|platform|platform_version| device|use_type_id|
+------+-------+--------+----------------+-------+-----------+
| 22845|  29658|     ios|             9.3|iPad2,4|          3|
| 22844|  29658|     ios|             9.3|iPad2,4|          3|
| 22846|  29658|     ios|             9.3|iPad2,4|          3|
+------+-------+--------+----------------+-------+-----------+
only showing top 3 rows



Select columns

In [20]:
device_df[['user_id','use_id']].show(5)

+-------+------+
|user_id|use_id|
+-------+------+
|  26980| 22782|
|  29628| 22783|
|  28473| 22784|
|  15200| 22785|
|  28239| 22786|
+-------+------+
only showing top 5 rows



Better use select method

In [23]:
device_df.select('user_id','use_id').show(5)

+-------+------+
|user_id|use_id|
+-------+------+
|  26980| 22782|
|  29628| 22783|
|  28473| 22784|
|  15200| 22785|
|  28239| 22786|
+-------+------+
only showing top 5 rows



Filtering

In [24]:
device_df.filter(device_df.platform == 'ios').show(5)

+------+-------+--------+----------------+---------+-----------+
|use_id|user_id|platform|platform_version|   device|use_type_id|
+------+-------+--------+----------------+---------+-----------+
| 22782|  26980|     ios|            10.2|iPhone7,2|          2|
| 22785|  15200|     ios|            10.2|iPhone7,2|          3|
| 22791|  28775|     ios|            10.2|iPhone6,2|          3|
| 22796|  29641|     ios|            10.2|iPhone7,2|          3|
| 22797|  29642|     ios|            10.1|iPhone5,2|          3|
+------+-------+--------+----------------+---------+-----------+
only showing top 5 rows



Filtering using where, which is an alias for filter (in the following, using filter in place of where also works

In [25]:
device_df.select('user_id','platform','platform_version').where("platform == 'ios' and platform_version like '%.1%'").show(5)

+-------+--------+----------------+
|user_id|platform|platform_version|
+-------+--------+----------------+
|  29642|     ios|            10.1|
|  29642|     ios|            10.1|
|  26668|     ios|            10.1|
|  26668|     ios|            10.1|
|  29653|     ios|            10.1|
+-------+--------+----------------+
only showing top 5 rows



In [26]:
device_df.where("platform == 'ios' and platform_version like '%.1%'").show(5)

+------+-------+--------+----------------+---------+-----------+
|use_id|user_id|platform|platform_version|   device|use_type_id|
+------+-------+--------+----------------+---------+-----------+
| 22797|  29642|     ios|            10.1|iPhone5,2|          3|
| 22798|  29642|     ios|            10.1|iPhone5,2|          3|
| 22825|  26668|     ios|            10.1|iPhone9,4|          2|
| 22826|  26668|     ios|            10.1|iPhone9,4|          2|
| 22829|  29653|     ios|            10.1|iPhone7,2|          2|
+------+-------+--------+----------------+---------+-----------+
only showing top 5 rows



Count number of rows

In [27]:
device_df.count()

272

Show statistics

In [28]:
usage_df.describe().show()

+-------+-----------------------+----------------------+------------------+------------------+
|summary|outgoing_mins_per_month|outgoing_sms_per_month|        monthly_mb|            use_id|
+-------+-----------------------+----------------------+------------------+------------------+
|  count|                    240|                   240|               240|               240|
|   mean|     274.55916666666667|     98.96829166666666|3628.6020416666647|23285.516666666666|
| stddev|     293.74574441310455|    111.17268546411792| 4486.311513407213|   624.13925309376|
|    min|                    0.5|                  0.25|                 0|             22787|
|    max|                 993.35|                 99.85|           9854.79|             25220|
+-------+-----------------------+----------------------+------------------+------------------+



Operations: subtract, union, intersect.

Here is a dataframe of use_id in device_df but not in usage_df

In [29]:
device_df.select('use_id').subtract(usage_df.select('use_id')).show(5)

+------+
|use_id|
+------+
| 22893|
| 22919|
| 22800|
| 22923|
| 22821|
+------+
only showing top 5 rows



Here is a dataframe of use_id in device_df or usage_df

In [30]:
device_df.select('use_id').union(usage_df.select('use_id')).show(5)

+------+
|use_id|
+------+
| 22782|
| 22783|
| 22784|
| 22785|
| 22786|
+------+
only showing top 5 rows



Here is a dataframe of use_id in both device_df and usage_df

In [31]:
device_df.select('use_id').intersect(usage_df.select('use_id')).show(5)

+------+
|use_id|
+------+
| 22920|
| 22831|
| 22922|
| 22875|
| 22895|
+------+
only showing top 5 rows



Inner join

In [32]:
device_df.join(usage_df, device_df.use_id == usage_df.use_id,"inner").select('*').show(5)

+------+-------+--------+----------------+--------+-----------+-----------------------+----------------------+----------+------+
|use_id|user_id|platform|platform_version|  device|use_type_id|outgoing_mins_per_month|outgoing_sms_per_month|monthly_mb|use_id|
+------+-------+--------+----------------+--------+-----------+-----------------------+----------------------+----------+------+
| 22787|  12921| android|             4.3|GT-I9505|          1|                  21.97|                  4.82|   1557.33| 22787|
| 22788|  28714| android|               6|SM-G930F|          1|                1710.08|                136.88|   7267.55| 22788|
| 22789|  28714| android|               6|SM-G930F|          1|                1710.08|                136.88|   7267.55| 22789|
| 22790|  29592| android|             5.1|   D2303|          1|                  94.46|                 35.17|    519.12| 22790|
| 22792|  28217| android|             5.1|SM-G361F|          1|                  71.59|          

Right join

In [33]:
device_df.join(usage_df, device_df.use_id == usage_df.use_id,"right_outer").select('*').show(5)

+------+-------+--------+----------------+--------+-----------+-----------------------+----------------------+----------+------+
|use_id|user_id|platform|platform_version|  device|use_type_id|outgoing_mins_per_month|outgoing_sms_per_month|monthly_mb|use_id|
+------+-------+--------+----------------+--------+-----------+-----------------------+----------------------+----------+------+
| 22787|  12921| android|             4.3|GT-I9505|          1|                  21.97|                  4.82|   1557.33| 22787|
| 22788|  28714| android|               6|SM-G930F|          1|                1710.08|                136.88|   7267.55| 22788|
| 22789|  28714| android|               6|SM-G930F|          1|                1710.08|                136.88|   7267.55| 22789|
| 22790|  29592| android|             5.1|   D2303|          1|                  94.46|                 35.17|    519.12| 22790|
| 22792|  28217| android|             5.1|SM-G361F|          1|                  71.59|          

Left join

In [34]:
device_df.join(usage_df, device_df.use_id == usage_df.use_id,"left_outer").select('*').show(5)

+------+-------+--------+----------------+---------+-----------+-----------------------+----------------------+----------+------+
|use_id|user_id|platform|platform_version|   device|use_type_id|outgoing_mins_per_month|outgoing_sms_per_month|monthly_mb|use_id|
+------+-------+--------+----------------+---------+-----------+-----------------------+----------------------+----------+------+
| 22782|  26980|     ios|            10.2|iPhone7,2|          2|                   null|                  null|      null|  null|
| 22783|  29628| android|               6|  Nexus 5|          3|                   null|                  null|      null|  null|
| 22784|  28473| android|             5.1| SM-G903F|          1|                   null|                  null|      null|  null|
| 22785|  15200|     ios|            10.2|iPhone7,2|          3|                   null|                  null|      null|  null|
| 22786|  28239| android|               6|ONE E1003|          1|                   null|  

Full outer join

In [35]:
device_df.join(usage_df, device_df.use_id == usage_df.use_id,"full_outer").select('*').show(5)

+------+-------+--------+----------------+---------+-----------+-----------------------+----------------------+----------+------+
|use_id|user_id|platform|platform_version|   device|use_type_id|outgoing_mins_per_month|outgoing_sms_per_month|monthly_mb|use_id|
+------+-------+--------+----------------+---------+-----------+-----------------------+----------------------+----------+------+
| 22893|  29672|     ios|            10.1|iPhone9,3|          3|                   null|                  null|      null|  null|
| 22920|  18833| android|               6|    D6603|          1|                 362.01|                 14.19|   7267.55| 22920|
| 22831|   6541| android|             4.1|GT-I8190N|          1|                  85.97|                 26.94|    407.01| 22831|
| 22919|  29679|     ios|            10.1|  iPad4,1|          3|                   null|                  null|      null|  null|
| 22922|  28845| android|               6|  LG-H815|          1|                 155.71|  

Convert to RDD to map. Note that you can always convert a DataFrame to RDD

In [36]:
device_df.rdd.map(lambda x:(x,1)).take(5)

[(Row(use_id='22782', user_id='26980', platform='ios', platform_version='10.2', device='iPhone7,2', use_type_id='2'),
  1),
 (Row(use_id='22783', user_id='29628', platform='android', platform_version='6', device='Nexus 5', use_type_id='3'),
  1),
 (Row(use_id='22784', user_id='28473', platform='android', platform_version='5.1', device='SM-G903F', use_type_id='1'),
  1),
 (Row(use_id='22785', user_id='15200', platform='ios', platform_version='10.2', device='iPhone7,2', use_type_id='3'),
  1),
 (Row(use_id='22786', user_id='28239', platform='android', platform_version='6', device='ONE E1003', use_type_id='1'),
  1)]

In [37]:
import pyspark.sql.functions as func
device_df.withColumn('platform_version_main_changes', func.round(device_df.platform_version,0)).show(3)

+------+-------+--------+----------------+---------+-----------+-----------------------------+
|use_id|user_id|platform|platform_version|   device|use_type_id|platform_version_main_changes|
+------+-------+--------+----------------+---------+-----------+-----------------------------+
| 22782|  26980|     ios|            10.2|iPhone7,2|          2|                         10.0|
| 22783|  29628| android|               6|  Nexus 5|          3|                          6.0|
| 22784|  28473| android|             5.1| SM-G903F|          1|                          5.0|
+------+-------+--------+----------------+---------+-----------+-----------------------------+
only showing top 3 rows

